<a href="https://colab.research.google.com/github/JounKK/MLstudy/blob/main/NLP_02_%EB%8B%A8%EC%96%B4%EB%B2%A1%ED%84%B0%ED%99%94_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬으로 TF-IDF 행렬구현

In [1]:
import pandas as pd
from math import log

In [15]:
# 4개의 문서
documents = [
    '탕수육 먹고 싶어요',
    '자장면도 먹고 싶어요',
    '탕수육과 자장면은 중국음식',
    '저는 탕수육이 좋아요'
]

# vocab 구축
vocab = list(set(w for document in documents for w in document.split()))

In [16]:
print(vocab)

['싶어요', '저는', '탕수육과', '탕수육이', '자장면은', '자장면도', '먹고', '탕수육', '좋아요', '중국음식']


In [17]:
# 총 문서 수
N = len(documents)
print('총 문서 수 : ', N)

총 문서 수 :  4


## <b>TF-IDF</b>
* TF (Term Frequency, 단어 빈도) : 특정 문서에서 단어가 몇 번 나왔니?
    * 예) '사과'라는 단어가 문서에 3번 나왔으면, TF = 3
* DF (Document Frequency, 문서 빈도) : 단어가 몇 개 문서에서 나왔니?
    * 예) '사과'가 5개의 문서에서 나왔으면, DF = 5
* IDF (Inverse Document Frequency, 역문서 빈도) : 단어가 얼마나 레어템이니?
    * 흔하면 점수가 낮고, 흔하지 않으면 점수가 높다
    * 각각이 여러 문서에서 어느정도 나왔는지 차이에 따라 흔한지 아닌지 알 수 있음.
    * 예) '사과' vs. '그' 이런 수식어 비교에서 사용가능할듯
* TF-IDF (Term Frequency-Inverse Document Frequency, 단어 중요도) : 특정 문서에서 단어가 얼마나 중요하니?
    * TF X IDF
    * 한 문서에서 TF가 높고(자주 등장하는데), IDF가 높으면(레어템이면) 높은 값이 나오고, 중요하다.

In [18]:
# TF를 구하는 함수
def tf(t, d):
    return d.count(t)

In [19]:
# IDF를 구하는 함수
def idf(t):
    df = 0
    for doc in documents:
        df += t in doc
    return log(N/(df+1))

In [20]:
# TF와 IDF의 값을 곱하는 함수
def tfidf(t, d):
    return tf(t,d)* idf(t)

* 위 문서 예시를 가지고 TF 구하기

In [21]:
result = []

# 각 문서에 대한 연산
for i in range(N):
    result.append([])
    d = documents[i]
    for j in range(len(vocab)):
        t = vocab[j]
        # tf함수 호출하여 더하기
        result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns = vocab)

In [22]:
tf_

,싶어요,저는,탕수육과,탕수육이,자장면은,자장면도,먹고,탕수육,좋아요,중국음식
0,1,0,0,0,0,0,1,1,0,0
1,1,0,0,0,0,1,1,0,0,0
2,0,0,1,0,1,0,0,1,0,1
3,0,1,0,1,0,0,0,1,1,0


* 위 문서 예시를 가지고 IDF 구하기

In [23]:
result = []

for j in range(len(vocab)):
    t = vocab[j]
    # idf 함수를 호출
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_

,IDF
싶어요,0.287682
저는,0.693147
탕수육과,0.693147
탕수육이,0.693147
자장면은,0.693147
자장면도,0.693147
먹고,0.287682
탕수육,0.000000
좋아요,0.693147
중국음식,0.693147


* TF-IDF 행렬을 구하기

In [24]:
result = []

for i in range(N):
    result.append([])
    d = documents[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,싶어요,저는,탕수육과,탕수육이,자장면은,자장면도,먹고,탕수육,좋아요,중국음식
0,0.287682,0.000000,0.000000,0.000000,0.000000,0.000000,0.287682,0.0,0.000000,0.000000
1,0.287682,0.000000,0.000000,0.000000,0.000000,0.693147,0.287682,0.0,0.000000,0.000000
2,0.000000,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,0.0,0.000000,0.693147
3,0.000000,0.693147,0.000000,0.693147,0.000000,0.000000,0.000000,0.0,0.693147,0.000000


# 사이킷런을 활용하기

### **BoW(Bag of Words)**

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    '강아지가 좋아요',
    '고양이가 좋아요, 더 좋아요',
    '고양이가 싫어요'
]

# CountVectorizer 객체 생성
vector = CountVectorizer()

# 텍스트 데이터를 수치 특성 벡터로 변환
x = vector.fit_transform(corpus)

# 어휘사전 확인
vocab = vector.get_feature_names_out()
print("어휘사전 : ", vocab)

# 코퍼스로부터 각 단어 빈도수 기록 확인
print(x.toarray())

# 단어별 인덱스 확인
print(vector.vocabulary_)

어휘사전 :  ['강아지가' '고양이가' '싫어요' '좋아요']
[[1 0 0 1]
 [0 1 0 2]
 [0 1 1 0]]
{'강아지가': 0, '좋아요': 3, '고양이가': 1, '싫어요': 2}


### **TF-IDF**

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    '강아지가 좋아요',
    '고양이가 좋아요, 더 좋아요',
    '고양이가 싫어요'
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.79596054 0.         0.         0.60534851]
 [0.         0.4472136  0.         0.89442719]
 [0.         0.60534851 0.79596054 0.        ]]
{'강아지가': 0, '좋아요': 3, '고양이가': 1, '싫어요': 2}


## 문서간 유사도 구하기

* **유클리드 거리**

    * 유클리드 거리 값이 작을수록 유사함
    * 그러나 이걸 이용하는 게 좋지 않은 결과를 보인다

In [40]:
from numpy.linalg import norm
import numpy as np
from numpy import dot

def euclidean_distance(A, B):
    return np.linalg.norm(A - B)

doc1 = np.array([1,1,0,0])
doc2 = np.array([3,3,0,0])
doc3 = np.array([1,0,1,1])

print("doc1의 단어수:", sum(doc1), "doc2의 단어수:", sum(doc2), "doc3의 단어수:", sum(doc3))

# 1 - 2 유클리드 거리 출력
print(euclidean_distance(doc1, doc2))
# 1 - 3 유클리드 거리 출력
print(euclidean_distance(doc1, doc3))

# 문장 길이에 영향을 받는다 !

doc1의 단어수: 2 doc2의 단어수: 6 doc3의 단어수: 3
2.8284271247461903
1.7320508075688772


* **코사인 유사도**
    * 결과가 1에 가까울수록 유사하다 (-1 ~ 1 사이 값을 가짐)
    * 두 벡터간의 각도를 이용하여 구함. 방향이 동일하면 1
    * 모든 단어의 빈도수가 동일하게 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이 된다.
        * 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교가능
        * 예) doc1과 doc2는 문장 길이가 3배 차이남, doc3이 문장 길이는 doc1과 더 비슷, 그러나 같은 내용이 많으므로 방향이 doc2가 더 비슷
        * 같은 주제라면 같은 단어들이 더 자주 등장하기 때문에 방향성을 고려하는 것이 비교에 용이

    * 참고 : https://wikidocs.net/24603

In [35]:
def cosine_similarity(A, B):
    return dot(A, B) / (norm(A) * norm(B))

# 문서 1과 문서 2의 코사인 유사도 출력
print(cosine_similarity(doc1, doc2))
# 문서 1과 문서 3의 코사인 유사도 출력
print(cosine_similarity(doc1, doc3))

1.0
0.40824829046386296
